# Pobieranie danych Eurostat

In [287]:
import eurostat

In [83]:
toc_df = eurostat.get_toc_df()

### Pobieranie metadanych o dostępnych datasetach

In [85]:
toc_df.to_excel("datasets_eurostat.xlsx", index=False)

In [86]:
eurostat.get_pars("MIPS_SA")

['freq', 'indic_ip', 'release', 'indic', 'geo']

In [103]:
dic = eurostat.get_dic("GOV_10DD_EDPT1", "sector")


In [204]:
gdp_data =  eurostat.get_data_df("namq_10_gdp",filter_pars = {"geo": ["PL"]})

In [254]:
selected_vars = ["B1GQ", "P3", 'P5G', 'P6', 'P7']

In [255]:
dic_item = eurostat.get_dic("namq_10_gdp", "na_item")
dic_unit = eurostat.get_dic("namq_10_gdp", "unit")

In [256]:
dict_item_dataset = {k : v for k, v in dict(dic_item).items() if k in gdp_data["na_item"].drop_duplicates().tolist()}
dict_unit_dataset = {k : v for k, v in dict(dic_unit).items() if k in gdp_data["unit"].drop_duplicates().tolist()}

In [257]:
selected_vars_dict = {k : v for k, v in dict(dic_item).items() if k in selected_vars}
selected_vars_dict

{'B1GQ': 'Gross domestic product at market prices',
 'P3': 'Final consumption expenditure',
 'P5G': 'Gross capital formation',
 'P6': 'Exports of goods and services',
 'P7': 'Imports of goods and services'}

In [258]:
quarter_start_month = {
    'Q1': '01',
    'Q2': '04',
    'Q3': '07',
    'Q4': '10'
}

In [285]:
eurostat_macro_df= gdp_data[gdp_data["na_item"].isin(selected_vars)].melt(id_vars = ["freq", "unit", "s_adj", "na_item", "geo\TIME_PERIOD"]).dropna(subset = ["value"])
eurostat_macro_df[['Year', 'Quarter']] = eurostat_macro_df["variable"].str.split('-', expand=True)
eurostat_macro_df['date'] = eurostat_macro_df['Year'] + '-' + eurostat_macro_df['Quarter'].map(quarter_start_month) + '-01'
eurostat_macro_df['date'] = pd.to_datetime(eurostat_macro_df['date'])
eurostat_macro_df = eurostat_macro_df[(eurostat_macro_df["date"].dt.year > 1999) & (eurostat_macro_df["s_adj"] == "NSA") & (eurostat_macro_df["unit"] == "CLV_PCH_SM")]
eurostat_macro_df.drop(columns = ["freq", "unit", "s_adj", "geo\TIME_PERIOD", "Quarter", "variable", "Year"], inplace = True)
eurostat_macro_df["na_item"] = eurostat_macro_df["na_item"].map(dict_item_dataset)

<>:1: SyntaxWarning: invalid escape sequence '\T'
<>:6: SyntaxWarning: invalid escape sequence '\T'
<>:1: SyntaxWarning: invalid escape sequence '\T'
<>:6: SyntaxWarning: invalid escape sequence '\T'
C:\Users\rados\AppData\Local\Temp\ipykernel_22984\3968699977.py:1: SyntaxWarning: invalid escape sequence '\T'
  eurostat_macro_df= gdp_data[gdp_data["na_item"].isin(selected_vars)].melt(id_vars = ["freq", "unit", "s_adj", "na_item", "geo\TIME_PERIOD"]).dropna(subset = ["value"])
C:\Users\rados\AppData\Local\Temp\ipykernel_22984\3968699977.py:6: SyntaxWarning: invalid escape sequence '\T'
  eurostat_macro_df.drop(columns = ["freq", "unit", "s_adj", "geo\TIME_PERIOD", "Quarter", "variable", "Year"], inplace = True)


In [286]:
eurostat_macro_df

na_item value       date
30630  Gross domestic product at market prices   6.2 2000-01-01
30631            Final consumption expenditure   4.7 2000-01-01
30632                  Gross capital formation   6.0 2000-01-01
30633            Exports of goods and services  19.9 2000-01-01
30634            Imports of goods and services  13.0 2000-01-01
...                                        ...   ...        ...
60826            Final consumption expenditure   4.8 2024-10-01
60827                  Gross capital formation   8.5 2024-10-01
60828            Exports of goods and services   0.5 2024-10-01
60829            Imports of goods and services   2.9 2024-10-01
61130  Gross domestic product at market prices   3.7 2025-01-01

[501 rows x 3 columns]

### Dane GUS

In [293]:
gus_raw.keys()


dict_keys(['Dane_miesięczne', 'Dane_kwartalne'])

In [301]:

gus_monthly = pd.read_excel("0_rawdata\gus_data.xlsx", sheet_name="Dane_miesięczne")
gus_quarterly =  pd.read_excel("0_rawdata\gus_data.xlsx", sheet_name='Dane_kwartalne', index_col=[0,1]).reset_index()

<>:1: SyntaxWarning: invalid escape sequence '\g'
<>:2: SyntaxWarning: invalid escape sequence '\g'
<>:1: SyntaxWarning: invalid escape sequence '\g'
<>:2: SyntaxWarning: invalid escape sequence '\g'
C:\Users\rados\AppData\Local\Temp\ipykernel_22984\656996444.py:1: SyntaxWarning: invalid escape sequence '\g'
  gus_monthly = pd.read_excel("0_rawdata\gus_data.xlsx", sheet_name="Dane_miesięczne")
C:\Users\rados\AppData\Local\Temp\ipykernel_22984\656996444.py:2: SyntaxWarning: invalid escape sequence '\g'
  gus_quarterly =  pd.read_excel("0_rawdata\gus_data.xlsx", sheet_name='Dane_kwartalne', index_col=[0,1]).reset_index()


In [332]:
quarter_2 = {
        '1' : '01',
        '2': '04',
        '3' : '07',
        '4' : '10'
}

In [341]:
gus_q_df = gus_quarterly.copy()
gus_q_df["Kwartał"] = gus_q_df["Kwartał"].str.replace(" kw.", "")
gus_q_df['date'] = gus_q_df['Rok'].astype(str) + '-' + gus_q_df['Kwartał'].astype(str).map(quarter_2) + '-01'
gus_q_df['date'] = pd.to_datetime(gus_q_df['date'])
gus_q_df.drop(columns = ["Kwartał", "Rok"], inplace = True)

In [355]:
gus_q_df

Podaż pieniądza M3 (stan w końcu okresu)  \
0                                      114,0   
1                                      120,3   
2                                      114,7   
3                                      111,9   
4                                      114,7   
..                                       ...   
96                                     106,6   
97                                     108,4   
98                                     106,5   
99                                     109,3   
100                                    110,3   

    Produkt krajowy brutto (ceny stale)c  \
0                                  106,5   
1                                  105,7   
2                                  103,4   
3                                  102,6   
4                                  102,4   
..                                   ...   
96                                101,9a   
97                                102,2a   
98                                102,3a   
99                                102,5a   
100                                  NaN   

     Oczekiwania inflacyjne przedsiębiorstw       date  
0                                       NaN 2000-01-01  
1                                       NaN 2000-04-01  
2                                       NaN 2000-07-01  
3                                       NaN 2000-10-01  
4                                       NaN 2001-01-01  
..                                      ...        ...  
96                                15.745192 2024-01-01  
97                                43.466828 2024-04-01  
98                                34.292663 2024-07-01  
99                                36.773795 2024-10-01  
100                               28.368794 2025-01-01  

[101 rows x 4 columns]

In [342]:
gus_q_df['date']

0     2000-01-01
1     2000-04-01
2     2000-07-01
3     2000-10-01
4     2001-01-01
         ...    
96    2024-01-01
97    2024-04-01
98    2024-07-01
99    2024-10-01
100   2025-01-01
Name: date, Length: 101, dtype: datetime64[ns]

In [354]:
gus_m_df = gus_monthly.copy()
gus_m_df['date'] = gus_m_df['Rok'].astype(str) + '-' + gus_m_df.astype(str)['Miesiąc']
gus_m_df['date'] = pd.to_datetime(gus_m_df['date'])
gus_m_df["Bieżący wskaźnik ufności konsumenckiej (BWUK)"].replace({".": pd.NA}, inplace = True)
gus_m_df.drop(columns = ["Rok", "Miesiąc"], inplace = True)
gus_m_df = gus_m_df.fillna(method="ffill")
gus_m_df

C:\Users\rados\AppData\Local\Temp\ipykernel_22984\7894805.py:6: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  gus_m_df = gus_m_df.fillna(method="ffill")


Stopa bezrobocia rejestrowanego  \
0                               13.7   
1                               14.0   
2                               14.0   
3                               13.8   
4                               13.6   
..                               ...   
307                              5.3   
308                              5.3   
309                              5.3   
310                              5.3   
311                              5.3   

     Przeciętne miesięczne nominalne wynagrodzenie brutto w sektorze przedsiębiorstwa  \
0                                                  NaN                                  
1                                                  NaN                                  
2                                                  NaN                                  
3                                                  NaN                                  
4                                                  NaN                                  
..                                                 ...                                  
307                                              107.7                                  
308                                              107.7                                  
309                                              107.7                                  
310                                              107.7                                  
311                                              107.7                                  

     Bieżący wskaźnik ufności konsumenckiej (BWUK)  \
0                                            -32.9   
1                                            -32.9   
2                                            -32.9   
3                                            -33.5   
4                                            -33.5   
..                                             ...   
307                                          -16.5   
308                                          -16.5   
309                                          -16.5   
310                                          -16.5   
311                                          -16.5   

    Wskaźniki cen towarów i usług konsumpcyjnych       date  
0                                          110,1 2000-01-01  
1                                          110,4 2000-02-01  
2                                          110,3 2000-03-01  
3                                          109,8 2000-04-01  
4                                          110,0 2000-05-01  
..                                           ...        ...  
307                                        104,9 2025-08-01  
308                                        104,9 2025-09-01  
309                                        104,9 2025-10-01  
310                                        104,9 2025-11-01  
311                                        104,9 2025-12-01  

[312 rows x 5 columns]

### Stooq data

In [399]:
usdpln_d = pd.read_csv("0_rawdata/usdpln_d.csv")
eurpln_d = pd.read_csv("0_rawdata/eurpln_d.csv")

In [428]:
eurpln_df = eurpln_d.copy()
usdpln_df = usdpln_d.copy()
eurpln_df["date"] = pd.to_datetime(eurpln_df["Data"])
eurpln_df.drop(columns = ["Data"], inplace = True)
usdpln_df["date"] = pd.to_datetime(usdpln_df["Data"])
usdpln_df.drop(columns = ["Data"], inplace = True)
usdpln_df["date"] = usdpln_df["date"].dt.to_period('M').dt.to_timestamp()
eurpln_df["date"] = eurpln_df["date"].dt.to_period('M').dt.to_timestamp()
stooq_df = usdpln_df.groupby("date")["Zamkniecie"].mean().to_frame("stock_USDPLN")
stooq_df["stock_EURPLN"] = eurpln_df.groupby("date")["Zamkniecie"].mean()
stooq_df.reset_index(inplace = True)
stooq_df = stooq_df[stooq_df["date"].dt.year > 1999]

In [429]:
stooq_df

date  stock_USDPLN  stock_EURPLN
192 2000-01-01      4.100857      4.153024
193 2000-02-01      4.143333      4.077543
194 2000-03-01      4.086935      3.942770
195 2000-04-01      4.236053      4.007063
196 2000-05-01      4.481452      4.069086
..         ...           ...           ...
492 2025-01-01      4.097845      4.244146
493 2025-02-01      4.005179      4.173267
494 2025-03-01      3.866003      4.181095
495 2025-04-01      3.801656      4.264490
496 2025-05-01      3.788023      4.257463

[305 rows x 3 columns]

### Oil Data

In [430]:
oil_raw = pd.read_csv("0_rawdata/Brent Oil Futures Historical Data (1).csv")

In [431]:
oil_df = oil_raw.copy()
oil_df["Date"] = pd.to_datetime(oil_df["Date"]) 
oil_df = oil_df.groupby("Date")["Price"].mean().to_frame("stock_brent_oil").reset_index()
oil_df = oil_df[oil_df["Date"].dt.year > 1999]
oil_df 

Date  stock_brent_oil
138 2000-01-01            25.97
139 2000-02-01            28.09
140 2000-03-01            24.77
141 2000-04-01            23.89
142 2000-05-01            28.31
..         ...              ...
438 2025-01-01            76.76
439 2025-02-01            73.18
440 2025-03-01            74.74
441 2025-04-01            63.12
442 2025-05-01            65.38

[305 rows x 2 columns]

# Merging datasets

## Quaterly datasets